In [1]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from himalaya.backend import set_backend
from matplotlib import pyplot as plt
import simplstyles

from residual_method import residual_method
from dataset import generate_dataset
from variance_partitioning import variance_partitioning

In [2]:
def get_path():
    path = os.path.join("results", "varying dimensions")
    os.makedirs(path, exist_ok=True)
    return path

In [3]:
def save_scores(d_list_list, n_samples=20000, n_samples_train=10000):
    path = get_path()
    for d_list in d_list_list:
        print(d_list)
        csv_path = os.path.join(path, f"scores_{d_list}.csv")
        scores = pd.DataFrame()
        if os.path.exists(csv_path):
            print("skipping, already exists")
            continue
        Xs, Y = generate_dataset(d_list=d_list, n_samples=n_samples)
        print("data generated")
        x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score = variance_partitioning(
            Xs, Y, n_samples_train)
        print("variance partitioning done")

        scores["x1_score"] = x1_score
        scores["x2_score"] = x2_score
        scores["vp_joint_score"] = joint_score
        scores["vp_shared_score"] = x1_and_x2_score
        scores["vp_x1_unique_score"] = vp_x1_unique_score
        scores["vp_x2_unique_score"] = vp_x2_unique_score
        del x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score

        x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score = residual_method(
            Xs, Y, n_samples_train)
        print("residual method done")
        scores["rm_x2_to_x1_score"] = np.concatenate(
            [x2_to_x1_score, np.full(len(rm_x1_unique_score) - len(x2_to_x1_score), np.nan)])
        scores["rm_x1_to_x2_score"] = np.concatenate(
            [x1_to_x2_score, np.full(len(rm_x1_unique_score) - len(x1_to_x2_score), np.nan)])
        scores["rm_x1_unique_score"] = rm_x1_unique_score
        scores["rm_x2_unique_score"] = rm_x2_unique_score
        del x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score

        # ridge residual
        x2_to_x1_ridge_score, x1_to_x2_ridge_score, rm_ridge_x1_unique_score, rm_ridge_x2_unique_score = residual_method(
            Xs, Y, n_samples_train, use_ols=False)
        print("residual method done")
        scores["rm_ridge_x2_to_x1_score"] = np.concatenate(
            [x2_to_x1_ridge_score, np.full(len(rm_ridge_x1_unique_score) - len(x2_to_x1_ridge_score), np.nan)])
        scores["rm_ridge_x1_to_x2_score"] = np.concatenate(
            [x1_to_x2_ridge_score, np.full(len(rm_ridge_x1_unique_score) - len(x1_to_x2_ridge_score), np.nan)])
        scores["rm_ridge_x1_unique_score"] = rm_ridge_x1_unique_score
        scores["rm_ridge_x2_unique_score"] = rm_ridge_x2_unique_score
        del x2_to_x1_ridge_score, x1_to_x2_ridge_score, rm_ridge_x1_unique_score, rm_ridge_x2_unique_score

        del Xs, Y
        scores.to_csv(csv_path, index=False)

# Save scores for varying Dimensions

In [4]:
backend = set_backend("torch_cuda", on_error="warn")
warnings.filterwarnings("ignore")
random.seed(42)
plt.style.use('nord-light-talk')

In [5]:
d_list = [100, 100, 100]

In [6]:
varying_dim = np.logspace(1, 3, 20, dtype=int)
varying_dim = list(map(int, varying_dim))

## Shared dimension

In [7]:
d_list_list = [[dim, d_list[1], d_list[2]] for dim in varying_dim]

In [ ]:
save_scores(d_list_list)

[10, 100, 100]


## Unique dimension

In [ ]:
d_list_list = [[d_list[0], int(dim), d_list[2]] for dim in varying_dim]

In [ ]:
save_scores(d_list_list)

# Plot scores

## Shared dimension

In [ ]:
vp_shared = pd.DataFrame()
rm_shared = pd.DataFrame()
rm_shared_ridge = pd.DataFrame()

for i, dim in enumerate(varying_dim):
    shared_dims = [dim, d_list[1], d_list[2]]
    scores = pd.read_csv(os.path.join(get_path(), f"scores_{shared_dims}.csv"))

    vp_shared_scores = pd.DataFrame({
        'shared_dim': [dim] * len(scores),
        'vp_x1_unique_score': scores['vp_x1_unique_score'],
    }, index=range(len(scores)))

    rm_shared_scores = pd.DataFrame({
        'shared_dim': [dim] * len(scores),
        'rm_x1_unique_score': scores['rm_x1_unique_score'],
    }, index=range(len(scores)))

    rm_shared_ridge_scores = pd.DataFrame({
        'shared_dim': [dim] * len(scores),
        'rm_ridge_x1_unique_score': scores['rm_ridge_x1_unique_score'],
    })

    vp_shared = pd.concat([vp_shared, vp_shared_scores], ignore_index=True)
    rm_shared = pd.concat([rm_shared, rm_shared_scores], ignore_index=True)
    rm_shared_ridge = pd.concat([rm_shared_ridge, rm_shared_ridge_scores], ignore_index=True)
rm_shared

In [ ]:
sns.lineplot(data=rm_shared_ridge, x='shared_dim', y='rm_ridge_x1_unique_score', label='Residual Method (Ridge)',
             errorbar='sd')
sns.lineplot(data=rm_shared, x='shared_dim', y='rm_x1_unique_score', label='Residual Method (OLS)', errorbar='sd')
sns.lineplot(data=vp_shared, x='shared_dim', y='vp_x1_unique_score', label='Variance Partitioning', errorbar='sd')
plt.axhline(0.3, linestyle='--', label='Expected Value', color="C3")
plt.xlabel(r"$d_\mathbf{A}$")
plt.ylabel(r"$R^2$ (mean and sd)")
plt.xscale('log')
plt.legend()

## Unique dimension

In [ ]:
vp_unique = pd.DataFrame()
rm_unique = pd.DataFrame()
rm_unique_ridge = pd.DataFrame()

for i, dim in enumerate(varying_dim):
    unique_dims = [d_list[0], dim, d_list[2]]
    scores = pd.read_csv(os.path.join(get_path(), f"scores_{unique_dims}.csv"))

    vp_unique_scores = pd.DataFrame({
        'unique_dim': [dim] * len(scores),
        'vp_x1_unique_score': scores['vp_x1_unique_score'],
    }, index=range(len(scores)))

    rm_unique_scores = pd.DataFrame({
        'unique_dim': [dim] * len(scores),
        'rm_x1_unique_score': scores['rm_x1_unique_score'],
    }, index=range(len(scores)))

    rm_unique_ridge_scores = pd.DataFrame({
        'unique_dim': [dim] * len(scores),
        'rm_ridge_x1_unique_score': scores['rm_ridge_x1_unique_score'],
    }, index=range(len(scores)))

    vp_unique = pd.concat([vp_unique, vp_unique_scores], ignore_index=True)
    rm_unique = pd.concat([rm_unique, rm_unique_scores], ignore_index=True)
    rm_unique_ridge = pd.concat([rm_unique_ridge, rm_unique_ridge_scores], ignore_index=True)

In [ ]:
sns.lineplot(data=rm_unique_ridge, x='unique_dim', y='rm_ridge_x1_unique_score', label='Residual Method (Ridge)',
             errorbar='sd')
sns.lineplot(data=rm_unique, x='unique_dim', y='rm_x1_unique_score', label='Residual Method (OLS)', errorbar='sd')
sns.lineplot(data=vp_unique, x='unique_dim', y='vp_x1_unique_score', label='Variance Partitioning', errorbar='sd')
plt.axhline(0.3, linestyle='--', label='Expected Value', color="C3")
plt.xlabel(r"$d_\mathbf{B}$")
plt.ylabel(r"$R^2$ (mean and sd)")
plt.xscale('log')
plt.legend()

In [ ]:
# add label for dotted lines
plt.plot([], [], linestyle='-', label=r'$d_\mathbf{A}$', color='black')
plt.plot([], [], linestyle=':', label=r'$d_\mathbf{B}$', color='black')

sns.lineplot(data=rm_shared_ridge, x='shared_dim', y='rm_ridge_x1_unique_score', label='Residual Method (Ridge)',
             errorbar='sd', color="C0")
sns.lineplot(data=rm_unique_ridge, x='unique_dim', y='rm_ridge_x1_unique_score', errorbar='sd', color="C0",
             linestyle=':')
sns.lineplot(data=rm_shared, x='shared_dim', y='rm_x1_unique_score', label='Residual Method (OLS)', errorbar='sd',
             color="C1")
sns.lineplot(data=rm_unique, x='unique_dim', y='rm_x1_unique_score', errorbar='sd', color="C1", linestyle=':')
sns.lineplot(data=vp_shared, x='shared_dim', y='vp_x1_unique_score', label='Variance Partitioning', errorbar='sd',
             color="C2", )
sns.lineplot(data=vp_unique, x='unique_dim', y='vp_x1_unique_score', errorbar='sd', color="C2", linestyle=':')


plt.axhline(0.3, linestyle='--', label='Expected Value', color="C3")
plt.xlabel(r"$d_\mathbf{A/B}$")
plt.ylabel(r"$R^2$ (mean and sd)")
plt.xscale('log')
plt.legend()